In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from streamlit_idealista.config import   INPUT_DATA_PATH, INPUT_OPERATION_TYPES_PATH, INPUT_TYPOLOGY_TYPES_PATH, INPUT_OPERATION_TYPES_PATH, INPUT_SUPERILLES_INTERVENTIONS_GEOJSON, INPUT_DTYPES_COUPLED_JSON_PATH, INPUT_INE_CENSUSTRACT_GEOJSON 
import functions as fc
#import streamlit as st
import folium as folium
from folium.plugins import Draw
from streamlit_folium import st_folium
from pathlib import Path
from shapely.geometry import GeometryCollection, shape
from shapely.ops import transform
from typing import Union,Optional,List
import numpy as np
import pandas as pd
from pathlib import Path
import json
import geopandas as gpd
import shapely
from prophet import Prophet
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pyproj import Transformer
from PIL import Image

2024-12-09 09:53:56.706 | INFO     | streamlit_idealista.config:<module>:11 - PROJ_ROOT path is: /Users/tomasandrade/Documents/BSC/idealista/streamlit-idealista
2024-12-09 09:53:56.707 | INFO     | streamlit_idealista.config:<module>:39 - Input data path: /Users/tomasandrade/Documents/BSC/idealista/streamlit-idealista/data/processed/full/02-metricas-de-mercado-extended-ad-2010-q2-2024_utf8_pivot.csv
2024-12-09 09:53:56.707 | INFO     | streamlit_idealista.config:<module>:40 - Input JSON path: /Users/tomasandrade/Documents/BSC/idealista/streamlit-idealista/data/processed/dtypes-coupled.json
2024-12-09 09:53:56.708 | INFO     | streamlit_idealista.config:<module>:41 - Input superilles interventions GeoJSON: /Users/tomasandrade/Documents/BSC/idealista/streamlit-idealista/data/processed/full/CENSUSTRACT_superilles.geojson
2024-12-09 09:53:57.604 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when r

In [2]:
import pyproj
pyproj.network.set_network_enabled(False)

In [52]:
# load data
def load_dtypes(dtypes_path: Path) -> dict:
    with open(dtypes_path, 'r') as f:
        return json.load(f)
dtypes_coupled_dict = load_dtypes(INPUT_DTYPES_COUPLED_JSON_PATH)

def load_main_data(main_data_path: Path) -> pd.DataFrame:
    return pd.read_csv( main_data_path, sep = ';', dtype=dtypes_coupled_dict)
df = load_main_data(INPUT_DATA_PATH)

def load_censustract_geojson(censustract_geojson_path: Path) -> gpd.GeoDataFrame:
    gdf_ine = gpd.read_file(censustract_geojson_path)
    gdf_ine['CENSUSTRACT'] = gdf_ine['CENSUSTRACT'].astype(int).astype(str)
    return gdf_ine
gdf_ine = load_censustract_geojson(INPUT_INE_CENSUSTRACT_GEOJSON)

def load_operation_types(operation_types_path: Path) -> pd.DataFrame:
    return pd.read_csv(operation_types_path, sep=";", dtype=dtypes_coupled_dict)
operation_types_df = load_operation_types(INPUT_OPERATION_TYPES_PATH)

def load_typology_types(typology_types_path: Path) -> pd.DataFrame:
    return pd.read_csv( typology_types_path, sep=";", dtype=dtypes_coupled_dict)
typology_types_df = load_typology_types(INPUT_TYPOLOGY_TYPES_PATH)

def load_interventions(interventions_path: Path) -> gpd.GeoDataFrame:
    return gpd.read_file(interventions_path)
interventions_gdf =  load_interventions(INPUT_SUPERILLES_INTERVENTIONS_GEOJSON)

def process_df(df: pd.DataFrame) -> pd.DataFrame:
    copy_df = df.copy(deep=True)
    return (
        copy_df
        .astype({'ADOPERATIONID': 'int',
                'ADTYPOLOGYID': 'int'
                })
        .join(operation_types_df.set_index('ID'), on='ADOPERATIONID', how="left", validate="m:1")
        .rename(columns={
            'SHORTNAME': 'ADOPERATION',
                        }
                )
        .astype({'ADOPERATION': 'category',
                'ADOPERATIONID': 'category'
                })
        .drop(columns=("DESCRIPTION"))
        .join(typology_types_df.set_index('ID'), on='ADTYPOLOGYID', how="left", validate="m:1")
        .rename(columns={
            'SHORTNAME': 'ADTYPOLOGY',
                        }
                )
        .astype({'ADTYPOLOGY': 'category',
                'ADTYPOLOGYID': 'category'
                })
        .drop(columns=("DESCRIPTION"))
    )
processed_df = process_df(df)

In [53]:
interventions_gdf = interventions_gdf.to_crs('EPSG:4326')

In [54]:
options=list(interventions_gdf["TITOL_WO"].unique())

In [55]:
geometry_selection = options[2:3]
geometry_selection

['Eixos Verds Eixample ']

In [100]:
# Add geometries to the layer
filtered_interventions_gdf = interventions_gdf[interventions_gdf["TITOL_WO"].isin(geometry_selection)]

processed_df['district'] = processed_df['CENSUSTRACT'].astype(str).str[4:6]
processed_df['munucipality'] = processed_df['CENSUSTRACT'].astype(str).str[0:4]

c1 = processed_df.district.isin(filtered_interventions_gdf.DISTRITO)
c2 = processed_df.munucipality.isin(filtered_interventions_gdf['PROVMUN'].astype(int).astype(str))

df_districts = processed_df[c1 & c2]

district_gdf = gdf_ine[gdf_ine.CENSUSTRACT.isin(df_districts.CENSUSTRACT)].to_crs('EPSG:4326')

In [101]:
# Create the base map
m = folium.Map(location=[41.40463, 2.17924], zoom_start=13, tiles="cartodbpositron")

# Create a GeoJSON layer for all geometries
geojson_layer = folium.FeatureGroup(name="Show Urban Interventions")

#for _, row in interventions_gdf.iterrows():
for _, row in filtered_interventions_gdf.iterrows():
    folium.GeoJson(
        row["geometry"],
        name=row["TITOL_WO"],
        tooltip = row["TITOL_WO"],
        style_function=lambda x: {
            "fillColor": "blue",
            "color": "blue",
            "weight": 2,
            "fillOpacity": 0.6,
        },
    ).add_to(geojson_layer)

for _, row in district_gdf.iterrows():
    folium.GeoJson(
        row["geometry"],
        style_function=lambda x: {
            "fillColor": "red",
            "color": "red",
            "weight": 1,
            "fillOpacity": 0.4,
        },
    ).add_to(geojson_layer)

# Add the GeoJSON layer to the map
geojson_layer.add_to(m)

# Add layer control to toggle visibility of geometries

# Initialize the Draw plugin
draw = Draw(
    draw_options={
        'polyline': False,
        'polygon': True,
        'rectangle': False,
        'circle': False,
        'marker': True,
    },
    edit_options={'edit': True},
)
draw.add_to(m)
folium.LayerControl(collapsed=False).add_to(m)
folium.plugins.Fullscreen(
    position="bottomleft",
    title="Expand me",
    title_cancel="Exit me",
    force_separate_button=True,
).add_to(m)
m
# Display the map in the Streamlit app
#output = st_folium(m, width=600, height=500)